In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
# linear modelling
import statsmodels.api as sm
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
# metrics
from sklearn.metrics import mean_squared_error
import seaborn as sns

# Census API Key
from census import Census
from config import api_key
c = Census(api_key, year=2017)

In [2]:
df=pd.read_csv("Airbnb_Texas_Rentals.csv")
df

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,3,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...
3,4,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...
4,5,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...
...,...,...,...,...,...,...,...,...,...,...
18254,18255,$60,1,Dallas,March 2013,An entire 1 bedroom 700+sqft condo in the hear...,32.892303,-96.772049,Quiet comfort living in Dallas,https://www.airbnb.com/rooms/1011576?location=...
18255,18256,$99,2,San Antonio,June 2015,An inviting 1920's cottage home in a popular u...,29.452893,-98.486756,Midtown Cottage Near Riverwalk,https://www.airbnb.com/rooms/18766940?location...
18256,18257,$13,1,Dallas,December 2016,Amazing 3BHK Apartment in a picturesque Commun...,33.001955,-96.777615,Room in Dallas!,https://www.airbnb.com/rooms/18719059?location...
18257,18258,$65,2,San Antonio,October 2016,My quaint and cozy home is conveniently locate...,29.450142,-98.505333,Vibrant Spacious Loft!,https://www.airbnb.com/rooms/18179329?location...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18259 entries, 0 to 18258
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              18259 non-null  int64  
 1   average_rate_per_night  18231 non-null  object 
 2   bedrooms_count          18256 non-null  object 
 3   city                    18259 non-null  object 
 4   date_of_listing         18259 non-null  object 
 5   description             18257 non-null  object 
 6   latitude                18225 non-null  float64
 7   longitude               18225 non-null  float64
 8   title                   18256 non-null  object 
 9   url                     18259 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.4+ MB


In [4]:
df.count()

Unnamed: 0                18259
average_rate_per_night    18231
bedrooms_count            18256
city                      18259
date_of_listing           18259
description               18257
latitude                  18225
longitude                 18225
title                     18256
url                       18259
dtype: int64

In [5]:
clean_df = df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18217 entries, 0 to 18258
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              18217 non-null  int64  
 1   average_rate_per_night  18217 non-null  object 
 2   bedrooms_count          18217 non-null  object 
 3   city                    18217 non-null  object 
 4   date_of_listing         18217 non-null  object 
 5   description             18217 non-null  object 
 6   latitude                18217 non-null  float64
 7   longitude               18217 non-null  float64
 8   title                   18217 non-null  object 
 9   url                     18217 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.5+ MB


In [6]:
# summary data frame
clean_df2 = clean_df.loc[:, ["city", "date_of_listing", "bedrooms_count", "average_rate_per_night", "latitude", "longitude"]]
clean_df2

,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude
0,Humble,May 2016,2,$27,30.020138,-95.293996
1,San Antonio,November 2010,4,$149,29.503068,-98.447688
2,Houston,January 2017,1,$59,29.829352,-95.081549
3,Bryan,February 2016,1,$60,30.637304,-96.337846
4,Fort Worth,February 2017,2,$75,32.747097,-97.286434
...,...,...,...,...,...,...
18254,Dallas,March 2013,1,$60,32.892303,-96.772049
18255,San Antonio,June 2015,2,$99,29.452893,-98.486756
18256,Dallas,December 2016,1,$13,33.001955,-96.777615
18257,San Antonio,October 2016,2,$65,29.450142,-98.505333


In [7]:
clean_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18217 entries, 0 to 18258
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    18217 non-null  object 
 1   date_of_listing         18217 non-null  object 
 2   bedrooms_count          18217 non-null  object 
 3   average_rate_per_night  18217 non-null  object 
 4   latitude                18217 non-null  float64
 5   longitude               18217 non-null  float64
dtypes: float64(2), object(4)
memory usage: 996.2+ KB


In [8]:
clean_df2["date_of_listing"]=pd.to_datetime(clean_df2["date_of_listing"])

clean_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18217 entries, 0 to 18258
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   city                    18217 non-null  object        
 1   date_of_listing         18217 non-null  datetime64[ns]
 2   bedrooms_count          18217 non-null  object        
 3   average_rate_per_night  18217 non-null  object        
 4   latitude                18217 non-null  float64       
 5   longitude               18217 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 996.2+ KB


In [9]:
clean_df2["year"]=[x.year for x in clean_df2.date_of_listing]
clean_df2.head()

,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude,year
0,Humble,2016-05-01,2,$27,30.020138,-95.293996,2016
1,San Antonio,2010-11-01,4,$149,29.503068,-98.447688,2010
2,Houston,2017-01-01,1,$59,29.829352,-95.081549,2017
3,Bryan,2016-02-01,1,$60,30.637304,-96.337846,2016
4,Fort Worth,2017-02-01,2,$75,32.747097,-97.286434,2017


In [10]:
unique_year = clean_df2["year"].nunique()
unique_year

10

In [11]:
clean_df2["month"]=[x.month for x in clean_df2.date_of_listing]
clean_df2.head()

,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude,year,month
0,Humble,2016-05-01,2,$27,30.020138,-95.293996,2016,5
1,San Antonio,2010-11-01,4,$149,29.503068,-98.447688,2010,11
2,Houston,2017-01-01,1,$59,29.829352,-95.081549,2017,1
3,Bryan,2016-02-01,1,$60,30.637304,-96.337846,2016,2
4,Fort Worth,2017-02-01,2,$75,32.747097,-97.286434,2017,2


In [12]:
clean_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18217 entries, 0 to 18258
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   city                    18217 non-null  object        
 1   date_of_listing         18217 non-null  datetime64[ns]
 2   bedrooms_count          18217 non-null  object        
 3   average_rate_per_night  18217 non-null  object        
 4   latitude                18217 non-null  float64       
 5   longitude               18217 non-null  float64       
 6   year                    18217 non-null  int64         
 7   month                   18217 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 1.3+ MB


In [13]:
mask1 = clean_df2.loc[clean_df2['year']==2012].index
mask2 = clean_df2.loc[clean_df2['year']==2011].index
mask3 = clean_df2.loc[clean_df2['year']==2010].index
mask4 = clean_df2.loc[clean_df2['year']==2009].index
mask5 = clean_df2.loc[clean_df2['year']==2008].index

In [14]:
mask1.array

<PandasArray>
[   12,    37,    89,   222,   262,   276,   311,   327,   340,   351,
 ...
 18090, 18120, 18136, 18137, 18146, 18159, 18184, 18205, 18213, 18215]
Length: 799, dtype: int64

In [15]:
clean_df3 = clean_df2[~(clean_df2.year.isin([2012, 2011, 2010, 2009, 2008]))].reset_index()
clean_df3.head()

,index,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude,year,month
0,0,Humble,2016-05-01,2,$27,30.020138,-95.293996,2016,5
1,2,Houston,2017-01-01,1,$59,29.829352,-95.081549,2017,1
2,3,Bryan,2016-02-01,1,$60,30.637304,-96.337846,2016,2
3,4,Fort Worth,2017-02-01,2,$75,32.747097,-97.286434,2017,2
4,5,Conroe,2016-08-01,4,$250,30.370455,-95.385319,2016,8


In [16]:
clean_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16799 entries, 0 to 16798
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   index                   16799 non-null  int64         
 1   city                    16799 non-null  object        
 2   date_of_listing         16799 non-null  datetime64[ns]
 3   bedrooms_count          16799 non-null  object        
 4   average_rate_per_night  16799 non-null  object        
 5   latitude                16799 non-null  float64       
 6   longitude               16799 non-null  float64       
 7   year                    16799 non-null  int64         
 8   month                   16799 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 1.2+ MB


In [17]:
# Create the bins in which Data will be held
# Bins are  12-1-2016, 7-1-2016, 1-1-2018
bins = [0, 3, 6, 9, 12]

# Create the names for the five bins
group_names = ["First", "Second", "Third", "Fourth"]

In [18]:
clean_df3["Date_Quarter"] = pd.cut(clean_df3["month"], bins, labels=group_names, include_lowest=True)
clean_df3

,index,city,date_of_listing,bedrooms_count,average_rate_per_night,latitude,longitude,year,month,Date_Quarter
0,0,Humble,2016-05-01,2,$27,30.020138,-95.293996,2016,5,Second
1,2,Houston,2017-01-01,1,$59,29.829352,-95.081549,2017,1,First
2,3,Bryan,2016-02-01,1,$60,30.637304,-96.337846,2016,2,First
3,4,Fort Worth,2017-02-01,2,$75,32.747097,-97.286434,2017,2,First
4,5,Conroe,2016-08-01,4,$250,30.370455,-95.385319,2016,8,Third
...,...,...,...,...,...,...,...,...,...,...
16794,18254,Dallas,2013-03-01,1,$60,32.892303,-96.772049,2013,3,First
16795,18255,San Antonio,2015-06-01,2,$99,29.452893,-98.486756,2015,6,Second
16796,18256,Dallas,2016-12-01,1,$13,33.001955,-96.777615,2016,12,Fourth
16797,18257,San Antonio,2016-10-01,2,$65,29.450142,-98.505333,2016,10,Fourth


In [19]:
# summary data frame
clean_df4 = clean_df3.loc[:, ["city", "year", "Date_Quarter", "bedrooms_count", "average_rate_per_night", "latitude", "longitude"]]
clean_df4.head()

,city,year,Date_Quarter,bedrooms_count,average_rate_per_night,latitude,longitude
0,Humble,2016,Second,2,$27,30.020138,-95.293996
1,Houston,2017,First,1,$59,29.829352,-95.081549
2,Bryan,2016,First,1,$60,30.637304,-96.337846
3,Fort Worth,2017,First,2,$75,32.747097,-97.286434
4,Conroe,2016,Third,4,$250,30.370455,-95.385319


In [20]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="http")

In [21]:
location = geolocator.reverse("30.020138, -95.293996")
location.raw

{'place_id': 291315935,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 198797061,
 'lat': '30.02013976623853',
 'lon': '-95.29412005281664',
 'display_name': '20521, Kenswick Drive, Bordersville, Harris County, Texas, 77338, United States',
 'address': {'house_number': '20521',
  'road': 'Kenswick Drive',
  'hamlet': 'Bordersville',
  'county': 'Harris County',
  'state': 'Texas',
  'postcode': '77338',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['30.020089766239',
  '30.020189766239',
  '-95.294170052817',
  '-95.294070052817']}

In [23]:
counties = []
zipcodes = []

for indx,row in clean_df4.iterrows():
    lat=row["latitude"]
    lon=row["longitude"]
    location = geolocator.reverse(f"{lat}, {lon}", timeout=None)
    
    county = location.raw["address"].get("county")
    zipcode = location.raw["address"].get("postcode")
    counties.append(county)
    zipcodes.append(zipcode)

    if indx % 100==0:
        print(indx)
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700


In [28]:
len(counties)

16799

In [29]:
clean_df4.head()


,city,year,Date_Quarter,bedrooms_count,average_rate_per_night,latitude,longitude
0,Humble,2016,Second,2,$27,30.020138,-95.293996
1,Houston,2017,First,1,$59,29.829352,-95.081549
2,Bryan,2016,First,1,$60,30.637304,-96.337846
3,Fort Worth,2017,First,2,$75,32.747097,-97.286434
4,Conroe,2016,Third,4,$250,30.370455,-95.385319


In [31]:
len(zipcodes)

16799

In [32]:
counties

['Harris County',
 'Harris County',
 'Brazos County',
 'Tarrant County',
 'Montgomery County',
 'Bastrop County',
 'Tarrant County',
 'Aransas County',
 'Bexar County',
 'Dallas County',
 'Tarrant County',
 'Bexar County',
 'Kerr County',
 'Guadalupe County',
 'Travis County',
 'Nueces County',
 'Harris County',
 'Denton County',
 'Harris County',
 'Brazos County',
 'Harris County',
 'Fort Bend County',
 'Denton County',
 'Fort Bend County',
 'Harris County',
 'Travis County',
 'Harris County',
 'Collin County',
 'Harris County',
 'Tarrant County',
 'Stephens County',
 'Harris County',
 'Travis County',
 'Tarrant County',
 'Dallas County',
 'Bell County',
 'Comal County',
 'Taylor County',
 'Harris County',
 'Taylor County',
 'Travis County',
 'Tarrant County',
 'Burnet County',
 'Harris County',
 'Brazos County',
 'Bexar County',
 'Collin County',
 'Bandera County',
 'Harris County',
 'Bexar County',
 'Harris County',
 'Tarrant County',
 'Fort Bend County',
 'Dallas County',
 'Travis 

In [44]:
clean_df4['counties'] = counties
clean_df4['zipcodes'] = zipcodes
clean_df4

,city,year,Date_Quarter,bedrooms_count,average_rate_per_night,latitude,longitude,counties,zipcodes
0,Humble,2016,Second,2,$27,30.020138,-95.293996,Harris County,77338
1,Houston,2017,First,1,$59,29.829352,-95.081549,Harris County,77049
2,Bryan,2016,First,1,$60,30.637304,-96.337846,Brazos County,77802
3,Fort Worth,2017,First,2,$75,32.747097,-97.286434,Tarrant County,76103
4,Conroe,2016,Third,4,$250,30.370455,-95.385319,Montgomery County,77303
...,...,...,...,...,...,...,...,...,...
16794,Dallas,2013,First,1,$60,32.892303,-96.772049,Dallas County,75230
16795,San Antonio,2015,Second,2,$99,29.452893,-98.486756,Bexar County,78212
16796,Dallas,2016,Fourth,1,$13,33.001955,-96.777615,Collin County,75252
16797,San Antonio,2016,Fourth,2,$65,29.450142,-98.505333,Bexar County,78212


In [ ]:
# new line here 